In [2]:
!pip install chess


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=23bfff1408a529ec27dc4d576806d61c2641ee427db800752055e990b083d5f2
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess


In [3]:
import chess
import chess.engine
import random
import numpy as np

In [4]:
# Step 1: Initialize Population
def initialize_population(size, chromosome_length):
    return [np.random.uniform(-1, 1, chromosome_length) for _ in range(size)]

In [5]:
# Step 2: Define Fitness Function
def evaluate_fitness(strategy, num_games=10):
    fitness = 0
    for _ in range(num_games):
        board = chess.Board()
        while not board.is_game_over():
            # AI plays as white
            move = choose_move(board, strategy)
            if move:
                board.push(move)
            else:
                break
            if board.is_game_over():
                break
            # Random move for the opponent
            legal_moves = list(board.legal_moves)
            if legal_moves:
                board.push(random.choice(legal_moves))
        # Evaluate fitness (e.g., win = +1, loss = -1, draw = 0)
        if board.result() == "1-0":  # AI wins
            fitness += 1
        elif board.result() == "0-1":  # AI loses
            fitness -= 1
    return fitness

In [6]:
# Step 3: Choose Move Based on Strategy
def choose_move(board, strategy):
    legal_moves = list(board.legal_moves)
    if not legal_moves:
        return None
    move_scores = []
    for move in legal_moves:
        board.push(move)
        score = evaluate_board(board, strategy)
        move_scores.append(score)
        board.pop()
    return legal_moves[np.argmax(move_scores)]

In [7]:
# Step 4: Evaluate Board Based on Strategy
def evaluate_board(board, strategy):
    material_score = sum(strategy[0] * (len(board.pieces(piece, chess.WHITE)) - len(board.pieces(piece, chess.BLACK)))
                         for piece in range(1, 7))
    mobility_score = strategy[1] * len(list(board.legal_moves))
    center_control_score = strategy[2] * sum(board.piece_at(chess.square(col, row)) is not None
                                             for col in range(3, 5) for row in range(3, 5))
    return material_score + mobility_score + center_control_score


In [8]:
# Step 5: Selection
def select_parents(population, fitnesses):
    total_fitness = sum(fitnesses)
    probabilities = [f / total_fitness for f in fitnesses]
    parents = random.choices(population, probabilities, k=2)
    return parents

In [9]:
# Step 6: Crossover
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

In [10]:
# Step 7: Mutation
def mutate(chromosome, mutation_rate=0.1):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            chromosome[i] += np.random.normal()
    return chromosome

In [11]:
# Step 8: Genetic Algorithm
def genetic_algorithm(pop_size, chromosome_length, num_generations):
    population = initialize_population(pop_size, chromosome_length)
    for generation in range(num_generations):
        fitnesses = [evaluate_fitness(individual) for individual in population]
        new_population = []
        for _ in range(pop_size // 2):
            parent1, parent2 = select_parents(population, fitnesses)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            new_population.extend([child1, child2])
        population = new_population
        best_fitness = max(fitnesses)
        print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")
    return population[np.argmax(fitnesses)]

In [13]:
# Step 9: Run the Genetic Algorithm
best_strategy = genetic_algorithm(pop_size=10, chromosome_length=3, num_generations=20)
print("Best Strategy:", best_strategy)

Generation 1, Best Fitness: 5
Generation 2, Best Fitness: -2
Generation 3, Best Fitness: -2
Generation 4, Best Fitness: 0
Generation 5, Best Fitness: -3
Generation 6, Best Fitness: -3
Generation 7, Best Fitness: -3
Generation 8, Best Fitness: -1
Generation 9, Best Fitness: -4
Generation 10, Best Fitness: -3
Generation 11, Best Fitness: -1
Generation 12, Best Fitness: -5
Generation 13, Best Fitness: 6
Generation 14, Best Fitness: -3
Generation 15, Best Fitness: -2
Generation 16, Best Fitness: -4
Generation 17, Best Fitness: -3
Generation 18, Best Fitness: -4
Generation 19, Best Fitness: -2
Generation 20, Best Fitness: -5
Best Strategy: [-1.27454679  0.8412636  -0.48413269]
